In [3]:

import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


iris = load_iris()
X = iris.data
y = iris.target


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


classes = np.unique(y_train)
mean = {}
var = {}
priors = {}

for c in classes:
    X_c = X_train[y_train == c]
    mean[c] = X_c.mean(axis=0)
    var[c] = X_c.var(axis=0)
    priors[c] = X_c.shape[0] / X_train.shape[0]


def gaussian_pdf(x, mean, var):
    eps = 1e-6  
    coeff = 1 / np.sqrt(2 * np.pi * var + eps)
    exponent = np.exp(- (x - mean) ** 2 / (2 * var + eps))
    return coeff * exponent


def predict(X):
    y_pred = []
    for x in X:
        posteriors = []
        for c in classes:
            prior = np.log(priors[c])
            conditional = np.sum(np.log(gaussian_pdf(x, mean[c], var[c])))
            posterior = prior + conditional
            posteriors.append(posterior)
        y_pred.append(classes[np.argmax(posteriors)])
    return np.array(y_pred)


y_pred = predict(X_test)
print("Step-by-Step", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=iris.target_names))


Step-by-Step 0.9777777777777777
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        19
  versicolor       1.00      0.92      0.96        13
   virginica       0.93      1.00      0.96        13

    accuracy                           0.98        45
   macro avg       0.98      0.97      0.97        45
weighted avg       0.98      0.98      0.98        45



In [4]:

from sklearn.naive_bayes import GaussianNB
iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


model = GaussianNB()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)

print("In-built ", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=iris.target_names))


In-built  0.9777777777777777
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        19
  versicolor       1.00      0.92      0.96        13
   virginica       0.93      1.00      0.96        13

    accuracy                           0.98        45
   macro avg       0.98      0.97      0.97        45
weighted avg       0.98      0.98      0.98        45



In [ ]:

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)


knn = KNeighborsClassifier()


param_grid = {'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15]}

grid = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

print("Best value of K:", grid.best_params_)
print("Best cross-validation accuracy:", grid.best_score_)


best_knn = grid.best_estimator_
y_pred = best_knn.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))
